In [52]:
import os
import json
import re

In [53]:
with open('dataset.json', 'r', encoding='utf-8') as f:
    text = json.load(f)

In [54]:
text[0]["input"]

'Ты парень, певец. Увлекаешься музыкой. Поешь на сцене. Любишь славу и популярность. Продолжи диалог:\nСобеседник: Что ты думаешь о политике и новостях?\nТы: Ох, политика - это такой зверь!\nСобеседник: Но она влияет на жизнь всех людей.\nТы: Да, но я предпочитаю оставаться на стороне. Я больше интересуюсь музыкой и развлечениями.\nСобеседник: Но ведь политические решения могут повлиять на музыкальную индустрию тоже.\nТы: Ну да, это правда. Но я всегда стараюсь сосредоточиться на своей музыке и создавать что-то вдохновенное.\nСобеседник: Но обрати внимание - политические лидеры могут быть источником вдохновения для своих поклонников.\nТы: Может быть, но я предпочитаю находить вдохновение в своем внутреннем мире и делиться этим с другими людьми через музыку.\nСобеседник: Понимаю, каждый имеет свои приоритеты. Но ты не беспокоишься, что политические ошибки могут сказаться на мире, в котором мы живем?\nТы: '

In [55]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)  # удаляем знаки препинания
    tokens = text.split()
    return tokens

In [56]:
tokenize(text[0]['output'])

['конечно',
 'я',
 'хочу',
 'чтобы',
 'мир',
 'был',
 'лучше',
 'но',
 'я',
 'верю',
 'что',
 'путем',
 'вдохновения',
 'через',
 'музыку',
 'и',
 'развлечения',
 'мы',
 'можем',
 'внести',
 'свой',
 'вклад',
 'в',
 'преображение',
 'нашего',
 'общества']

In [57]:
words = set()
tokens = []

for x in text:
    inp = x['input']
    out = x['output']
    tokens += tokenize(inp)
    tokens += tokenize(out)

tokens = set(tokens)
len(tokens)

44114

In [59]:
words = sorted(list(tokens))
words += ['<stoppred>']
vocab_size = len(words)
print(' '.join(words))
print(vocab_size)

0 000 004 008 03092012 0314 065 069 1 10 100 1000 10000 100000руб 100200 1012 1015 1050 106 108 108мегапиксельной 10м 11 1100 112 1134 12 120 121 1294 13 130 135 1371 1392000 14 1413 141679111032 142 15 150 1500 152 1520 15го 16 1621 16281694 1661 1676 17 17481795 1770 1789 17лет 18 1802 1804 18181879 1835 1842 1844 1851 1857 1860 1862 1863 1864 1869 1870 1877 1878 1882 1885 1886 1889 189 18го 19 1900 1903 1906 19070000112220000014615 1910 1911 1912 1913 1914 1917 1918 19181919 19181925 19181929 1920х 1922 1924 1928 1929 19291930 1930 1936 19361937 1937 1941 1945 1946 1947 1953 1956 1957 1958 1959 196 1960 1960х 1970 1970х 1972 1973 1975 1976 1977 1980х 1984 1986 1987 1989 1990 1990х 1991 1992 1994 1995 1997 1998 1999 1м 2 20 200 2000 2000х 2001 2002 2003 200350 2004 2005 2007 2009 2010 2011 2012 2013 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 203 2030 2031 20х40 21 22 220 225 227 228 23 235 23года 24 2448 25 2500летняя 2580 26 27 28 29 297 2in1 2x 2дюймовые 2е 2их 3 30 300

In [60]:
stoi = { w:i for i, w in enumerate(words) }
itos = { i:w for i, w in enumerate(words) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ' '.join([itos[i] for i in l])

print(encode(tokenize("как дела")))
print(decode(encode(tokenize("как дела"))))

[13625, 8370]
как дела


In [61]:
text[0]['input']

'Ты парень, певец. Увлекаешься музыкой. Поешь на сцене. Любишь славу и популярность. Продолжи диалог:\nСобеседник: Что ты думаешь о политике и новостях?\nТы: Ох, политика - это такой зверь!\nСобеседник: Но она влияет на жизнь всех людей.\nТы: Да, но я предпочитаю оставаться на стороне. Я больше интересуюсь музыкой и развлечениями.\nСобеседник: Но ведь политические решения могут повлиять на музыкальную индустрию тоже.\nТы: Ну да, это правда. Но я всегда стараюсь сосредоточиться на своей музыке и создавать что-то вдохновенное.\nСобеседник: Но обрати внимание - политические лидеры могут быть источником вдохновения для своих поклонников.\nТы: Может быть, но я предпочитаю находить вдохновение в своем внутреннем мире и делиться этим с другими людьми через музыку.\nСобеседник: Понимаю, каждый имеет свои приоритеты. Но ты не беспокоишься, что политические ошибки могут сказаться на мире, в котором мы живем?\nТы: '

<h1>ПРЕПРОЦЕССИНГ</h1>

In [77]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm import tqdm

In [78]:
def preprocess_dialogs(text):
    dialogs = []
    for x in text:
        question, answer = tokenize(x['input']), tokenize(x['output'])

        encoded_q = encode(question)
        encoded_a = encode(answer)
        stop = encode(['<stoppred>'])
        dialogs.extend(encoded_q + encoded_a + stop)

    return torch.tensor(dialogs, dtype=torch.long)

In [79]:
data = preprocess_dialogs(text)
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [80]:
train_data[0:2]

tensor([39245, 24328])

<h1>ГИПЕРПАРАМЕТРЫ</h1>

In [95]:
# ---------------------------------------------------------
batch_size = 32
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 4
n_layer = 4
dropout = 0.2
checkpoint_interval = 500
# ---------------------------------------------------------

<h1>МОДЕЛЬ</h1>

In [96]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    if len(data) < block_size:
        raise ValueError(f"Data length ({len(data)}) is smaller than block_size ({block_size}).")
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens=100):
        stop_token = stoi['<stoppred>']
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            
            if idx_next.item() == stop_token:
                break
                
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [97]:
model = BigramLanguageModel().to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")
print(f"Using device: {device}")

Model parameters: 25.85M
Using device: cuda


In [98]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [99]:
for iter in tqdm(range(max_iters), desc="Training"):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Iter {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
    
    if iter % checkpoint_interval == 0 and iter > 0:
        torch.save({
            'model_state_dict': model.state_dict(),
            'stoi': stoi,
            'itos': itos,
            'vocab_size': vocab_size,
        }, f'llm_V1_check.pth')
    
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

torch.save({
    'model_state_dict': model.state_dict(),
    'stoi': stoi,
    'itos': itos,
    'vocab_size': vocab_size,
}, 'llm_V1_final.pth')
print("Model saved as 'llm1_V1_final.pth'")


raining:   0%|                                                                               | 0/5000 [00:01<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.35 GiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.71 GiB is allocated by PyTorch, and 42.48 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)